In [1]:
import requests
import json
import numpy as np
from bs4 import BeautifulSoup

In [5]:
from lxml import html
import pandas as pd
from urllib.request import Request, urlopen

In [9]:
from fake_useragent import UserAgent
ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]

In [10]:
def getProx():
    proxies_req = Request('https://www.sslproxies.org/')
    proxies_req.add_header('User-Agent', ua.random)
    proxies_doc = urlopen(proxies_req).read().decode('utf8')
    soup = BeautifulSoup(proxies_doc, 'html.parser')
    proxies_table = soup.find(id='proxylisttable')
         # Save proxies in the array
    global proxies
    proxies=[]
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append({
        'ip':   row.find_all('td')[0].string,
        'port': row.find_all('td')[1].string
      })
    print(len(proxies))
    print(proxies)

In [11]:
getProx()

100
[{'ip': '139.99.66.152', 'port': '3128'}, {'ip': '139.59.66.235', 'port': '3128'}, {'ip': '46.246.40.32', 'port': '3128'}, {'ip': '191.55.8.11', 'port': '3128'}, {'ip': '36.80.30.115', 'port': '8080'}, {'ip': '192.99.207.23', 'port': '40067'}, {'ip': '95.31.245.50', 'port': '59575'}, {'ip': '195.88.16.155', 'port': '35796'}, {'ip': '116.212.129.58', 'port': '42965'}, {'ip': '195.144.209.214', 'port': '46900'}, {'ip': '118.174.131.241', 'port': '49833'}, {'ip': '60.243.246.94', 'port': '60309'}, {'ip': '5.206.233.106', 'port': '40557'}, {'ip': '203.124.47.98', 'port': '63141'}, {'ip': '119.82.252.71', 'port': '39309'}, {'ip': '103.194.192.37', 'port': '38835'}, {'ip': '109.229.27.81', 'port': '53281'}, {'ip': '45.120.119.202', 'port': '54179'}, {'ip': '36.66.121.111', 'port': '55016'}, {'ip': '176.235.99.114', 'port': '61517'}, {'ip': '200.94.140.50', 'port': '30198'}, {'ip': '31.129.161.43', 'port': '52822'}, {'ip': '123.16.188.2', 'port': '48406'}, {'ip': '91.137.189.48', 'port': 

In [13]:
import random
proxy_index = random.randint(0, len(proxies) - 1)
proxy = proxies[proxy_index] 
req = Request('http://icanhazip.com')
req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

In [4]:
df =pd.read_csv("altmetricIDS.csv", header=None)

In [5]:
df.columns = ['Altmetrics_Id']
df.head()

,Altmetrics_Id
0,702709
1,403270
2,2394296
3,632123
4,411425


In [6]:
df["youtube_links"] = ""
df.head()

,Altmetrics_Id,youtube_links
0,702709,
1,403270,
2,2394296,
3,632123,
4,411425,


In [7]:
ids = np.array(df['Altmetrics_Id'])
ids[5255]

1969481

In [ ]:
alt_param  = {"key" : ""}

In [12]:
all_alt_data = requests.get("https://api.altmetric.com/v1/id/1969481", alt_param )

In [13]:
alt_data = json.loads(all_alt_data.text)
alt_data.keys()

dict_keys(['title', 'doi', 'tq', 'altmetric_jid', 'issns', 'journal', 'cohorts', 'context', 'authors', 'type', 'altmetric_id', 'schema', 'is_oa', 'cited_by_fbwalls_count', 'cited_by_feeds_count', 'cited_by_gplus_count', 'cited_by_msm_count', 'cited_by_posts_count', 'cited_by_qna_count', 'cited_by_rdts_count', 'cited_by_tweeters_count', 'cited_by_videos_count', 'cited_by_wikipedia_count', 'cited_by_accounts_count', 'last_updated', 'score', 'history', 'url', 'added_on', 'published_on', 'subjects', 'scopus_subjects', 'readers', 'readers_count', 'images', 'details_url'])

In [22]:
y_links = []
html = requests.get('https://www.altmetric.com/details/' + str(403270) + '/video', alt_param).text
bs = BeautifulSoup(html)
possible_links = bs.find_all('a')
for link in possible_links:
    if link.has_attr('href') and 'youtube' in link.attrs['href']:
        y_links.append(link.attrs['href'])
y_links

['https://www.youtube.com/watch?v=2iqRF6mWj-A',
 'https://www.youtube.com/watch?v=42lNiOVFrhc',
 'https://www.youtube.com/watch?v=2Oag2IpeUjc',
 'https://www.youtube.com/watch?v=7Csem3LocpQ']

In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common import exceptions

In [114]:
chrome = webdriver.Chrome('C:/ProgramData/chocolatey/bin/chromedriver.exe')

In [115]:
chrome.get('https://www.altmetric.com/details/403270/video')

In [116]:
def get_all_pages(main_link):
    myset= set()
    chrome.get(main_link)
    elements = chrome.find_elements_by_xpath("//a[@href]")
    for elem in elements:
        if "video/page:" in elem.get_attribute("href"):
            myset.add(elem.get_attribute("href"))
    return myset

In [117]:
def get_youtube_ids(link):
    y_links = []
    chrome.get(link)
    elements = chrome.find_elements_by_xpath("//a[@href]")
    for elem in elements:
        if "youtube" in elem.get_attribute("href"):
            y_links.append(elem.get_attribute("href")[32:])
    return y_links

In [131]:
i=6
for alt_id in ids[6:7]:
    y_links = get_youtube_ids('https://www.altmetric.com/details/' + str(alt_id) + '/video')
    myset = get_all_pages('https://www.altmetric.com/details/' + str(alt_id) + '/video')
    for link in myset:
        y_links.extend(get_youtube_ids(link))
    df.at[i, 'youtube_links'] = y_links
    i = i+1

In [132]:
df[0:8]

,Altmetrics_Id,youtube_links
0,702709,"[GSqdhz--yR8, FXvGFvspzyE, kL4C4gtlw18, GOUNrH..."
1,403270,"[2iqRF6mWj-A, 42lNiOVFrhc, 2Oag2IpeUjc, 7Csem3..."
2,2394296,"[gwueRq8VECo, k0HKvl5dYV0, j8eAMWh9UC8, PHLHtS..."
3,632123,"[GpfLHrtdcS4, YjRjtxFV8qQ, m-VMYWtS7vY, ZlW2qz..."
4,411425,"[2iqRF6mWj-A, i8RFibYUBbA, LkfqGZ1pLp8, xp0b3m..."
5,612624,"[1c1Z1XOW9l4, zAi6Qe0qiIs, Hxq4jNx6DiA, nyoc5e..."
6,193445,"[yNAXGacSOJQ, 8RubMaTFuAA, rbmHQOZjV5Q, Krd-SU..."
7,1294194,


In [125]:
len(df.loc[4, 'youtube_links'])

395

In [134]:
df.to_csv("youtube_links.csv")

In [14]:
alt_data['details_url']

'http://www.altmetric.com/details.php?citation_id=1969481'

In [9]:
def add_video(alt_id):
    try:
        response = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))
        if 'video' in dict(json.loads(response.content)):
            return dict(json.loads(response.content))['video']
    except:
        return ''

In [10]:
param = {"key" : ""}

In [11]:
url_content = "https://www.googleapis.com/youtube/v3/videos?part=snippet&key=&id="

In [12]:
url = "https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics"

In [ ]:
for alt_id in ids[5655:5755]:
    mylist = []
    alt_req = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))
    alt_rs= json.loads(alt_req.text)
    mylist.append({str(alt_id) : alt_rs})
    y_links = []
    html = requests.get('https://www.altmetric.com/details/' + str(alt_id) + '/video').text
    bs = BeautifulSoup(html)
    possible_links = bs.find_all('a')
    for link in possible_links:
        if link.has_attr('href') and 'youtube' in link.attrs['href']:
            y_links.append(link.attrs['href'])
    youtube_data = []
    for link in y_links:
        param["id"] = link[32:]
        response_statistics = requests.get(url, param)
        print(response_statistics.status_code)
        json_statistics = json.loads(response_statistics.text)
        youtube_data.append(json_statistics)
    mylist.append({'youtube':youtube_data})
    file = open(str(alt_id) + ".txt", "w+")
    json.dump(mylist, file)
    file.close()

In [22]:
import os
path = "C:/Users/srika/Youtube_dataSet"
files = os.listdir(path)
len(path)

30

In [33]:
for i in range(len(files)):
    files[i] = os.path.abspath(path + "/" + files[i])

In [37]:
len("C:\\Users\\srika\\Youtube_dataSet\\")

31

In [46]:
df_ = pd.DataFrame()
col = ['cited_by_fbwalls_count', 'cited_by_feeds_count', 'cited_by_gplus_count', 'cited_by_msm_count', 
       'cited_by_posts_count', 'cited_by_rdts_count', 'cited_by_tweeters_count', 'cited_by_videos_count']

In [47]:
stats = ['viewCount', 'likeCount',  'dislikeCount', 'favoriteCount','commentCount']

In [ ]:
i=0
for file in files:
    json_file = open(file, "r")
    data = json.load(json_file)
    df_.loc[i, "altmetric_id"] = file[31:-4]
    for c in col:
        try:
            df_.loc[i, c] = data[0][file[31:-4]][c]
        except KeyError:
            df_.loc[i, c] = 0
    i += 1
    json_file.close
    
df_.head() 

In [58]:
df_videos = pd.DataFrame()

In [ ]:
i=0
for file in files:
    json_file = open(file, "r")
    data = json.load(json_file)
    for j in range(len(data[1]['youtube'])):
        for stat in stats:
            try:
                df_videos.loc[i, 'altmetric_id'] = file[31:-4]
                df_videos.loc[i, stat] = data[1]['youtube'][j]['items'][0]['statistics'][stat]
            except IndexError:
                df_videos.loc[i, stat] = 0
            except KeyError:
                df_videos.loc[i, stat] = 0
        i += 1
    json_file.close()
df_videos.head()

In [60]:
df_.to_csv("altmetrics_data.csv")

In [61]:
df_videos.to_csv("videos.csv") 

In [ ]:
y_link = url_statistics +"GSqdhz--yR8"
rs_st = requests.get(y_link)
rs_st_d = json.loads(rs_st.text)
rs_st_d

In [ ]:
y_content = url_content + "GSqdhz--yR8"
rs_ct = requests.get(y_link)
rs_ct_d = json.loads(rs_st.text)
rs_ct_d

In [ ]:
social_media_columns = ['cited_by_fbwalls_count', 'cited_by_gplus_count', 'cited_by_msm_count', 'cited_by_posts_count', 
                        'cited_by_tweeters_count', 'cited_by_videos_count', 'cited_by_wikipedia_count', 
                        'cited_by_accounts_count',]

In [ ]:
cols = ['title', 'doi']

In [ ]:
for i in range(len(ids[:100])):
    rs = requests.get("https://api.altmetric.com/v1/id/" + str(ids[i]))
    if (rs.status_code == 200):
        rs_json = json.loads(rs.text)
        for col in cols:
            try:
                df.loc[i, col] = rs_json[col]
            except KeyError:
                df.loc[i, col] = 0

In [ ]:
df.head()

In [ ]:
df[:100].to_csv(path_or_buf = ".\\sample_dataset.csv", index=False)

In [ ]:
url_comments = "https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=_qUhIZcxNvE&key="

In [ ]:
response = requests.get(url)
print(response.status_code)

In [ ]:
json_data = json.loads(response.text)
json_data.keys()

In [ ]:
json_data["items"][2]['snippet']['topLevelComment']['snippet']

In [ ]:
lc = 0
dc = 0
for j in len(json_data['items']):
    lc + = json_data['items'][j]["likeCount"]
    dc + = json_data['items'][j]["likeCount"]

In [ ]:
# add author information
def add_all_info(alt_id):
    # create the dict
    result = defaultdict(dict)
    try:
        # make the request
        response = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))

        # add the values to the dict
        if 'authors' in dict(json.loads(response.content)):
            result['author_count'] = len(dict(json.loads(response.content))['authors'])
            result['authors'] = ', '.join(dict(json.loads(response.content))['authors'])
        else:
            result['author_count'] = ''
            result['authors'] = ''

         # add the values to the dict
        if 'doi' in dict(json.loads(response.content)):
            result['doi'] = dict(json.loads(response.content))['doi']
        else:
            result['doi'] = ''

        # add the abstract
        if 'abstract' in dict(json.loads(response.content)):
            result['abstract'] = dict(json.loads(response.content))['abstract']
        else:
            result['abstract'] = ''

        # add the pubdate
        if 'published_on' in dict(json.loads(response.content)):
            result['pub_date'] = dict(json.loads(response.content))['published_on']
        else:
            result['pub_date'] = ''

        # add the twitter posts count as of 2018/19
        if 'cited_by_tweeters_count' in dict(json.loads(response.content)):
            result['twitter_count_y18'] = dict(json.loads(response.content))['cited_by_tweeters_count']
        else:
            result['twitter_count_y18'] = ''

        # add the title of the article
        if 'title' in dict(json.loads(response.content)):
            result['title'] =  dict(json.loads(response.content))['title']
        else:
            result['title'] = ''

        # return the result
        return result
    except:
        return result

In [ ]:
r = add_all_info(702709 )

In [ ]:
r.keys()

In [15]:
rs_ = requests.get("https://api.altmetric.com/v1/id/" + str(2394296))

In [16]:
rs_js = json.loads(rs_.text)

In [18]:
rs_js.keys()

dict_keys(['title', 'doi', 'pmid', 'tq', 'altmetric_jid', 'issns', 'journal', 'cohorts', 'abstract', 'abstract_source', 'context', 'authors', 'type', 'altmetric_id', 'schema', 'is_oa', 'publisher_subjects', 'cited_by_fbwalls_count', 'cited_by_feeds_count', 'cited_by_gplus_count', 'cited_by_msm_count', 'cited_by_posts_count', 'cited_by_rdts_count', 'cited_by_tweeters_count', 'cited_by_videos_count', 'cited_by_wikipedia_count', 'cited_by_accounts_count', 'last_updated', 'score', 'history', 'url', 'added_on', 'published_on', 'subjects', 'readers', 'readers_count', 'images', 'details_url'])

In [ ]:
fl = open("temp.txt", 'w+')

In [ ]:
json.dump(rs_js, fl)

In [ ]:
rs_js['cited_by_accounts_count']

In [ ]:
rs_js['cited_by_videos_count']

In [ ]:
response_statistics = requests.get(url_statistics)
print(response.status_code)

In [ ]:
json_statistics = json.loads(response_statistics.text)
json_statistics.keys()

In [ ]:
json_statistics['items']